[View in Colaboratory](https://colab.research.google.com/github/cmajorsolo/machine_learning_python/blob/master/RNN_study_with_Tensorflow.ipynb)

In [7]:
import tensorflow as tf
x1 = tf.constant(5)
x2 = tf.constant(6)
# result = x1 * x2
result = tf.multiply(x1, x2)
print(result)

with tf.Session() as sess:
  output = sess.run(result)
print(output)


Tensor("Mul_9:0", shape=(), dtype=int32)


In [5]:
'''
build a neural network model
input > weight >hidden layer 1 (activation function) > weights > hidden layer 2(activation function) > weights > output layer

compare output to intended output > cost of loss function(corss entropy)
optimization function (optimizer) > minimize cost (AdamOptimizer....SGD, AdaGrad)

backpropagation

feed forward + backprop = epoch
'''

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data', one_hot = True)

n_nodes_hl1 = 500  #500 nodes on hidden layer 1
n_nodes_hl2 = 500  #500 nodes on hidden layer 2
n_nodes_hl3 = 500  #500 nodes on hidden layer 3

n_classes = 10  #10 numbers
batch_size = 100  #batch is the feature size. feed in one batch, update weights then repeat. 

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
  # (input_data * weights) + biases
  # if input data is 0, input_data * weights will be 0. No neurals will be fired
  # adding biases here to fire the neurons in this case
  hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])), 
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
  hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])), 
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
  hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])), 
                    'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
  output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])), 
                    'biases': tf.Variable(tf.random_normal([n_classes]))}
  # model for each layer (input_data * weights) + biases
  l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])  #matrix multiplication
  l1 = tf.nn.relu(l1)  #activation function
  
  l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])  #matrix multiplication
  l2 = tf.nn.relu(l2)  #activation function
  
  l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])  #matrix multiplication
  l2 = tf.nn.relu(l3)  #activation function

  output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
  
  return output

def train_neural_network(x):
  prediction = neural_network_model(x)
  cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
  
  # learning_rate = 0.001
  optimizer = tf.train.AdamOptimizer().minimize(cost)
  
  # cycles feed forward + backprop
  hm_epochs = 10
  
  with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    for epoch in range(hm_epochs):
      epoch_loss = 0
      for _ in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y}) # c is the cost
        epoch_loss += c
      print('Epoch', epoch, 'completed out of ', hm_epochs, 'loss:', epoch_loss)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))  #argmx returns max value
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)    
      

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch 0 completed out of  10 loss: 2739163.952392578
Epoch 1 completed out of  10 loss: 654612.3142814636
Epoch 2 completed out of  10 loss: 380246.40410614014
Epoch 3 completed out of  10 loss: 250210.49446150474
Epoch 4 completed out of  10 loss: 164160.6193971634
Epoch 5 completed out of  10 loss: 116557.556773711
Epoch 6 completed out of  10 loss: 93931.12151110172
Epoch 7 completed out of  10 loss: 70697.65365123749
Epoch 8 completed out of  10 loss: 53436.427868396044
Epoch 9 completed out of  10 loss: 45325.36055856943
Accuracy: 0.9574
